# Lab 10.1: Reproductive Strategy Calculator
## Chapter 10: The Reproduction Revolution

### 🎯 Learning Objectives
- Calculate and compare r vs K reproductive strategies
- Analyze trade-offs between offspring number and quality
- Predict optimal strategies for different environments
- Evaluate fitness consequences of reproductive decisions
- Apply life history theory to real species

### 📖 Connection to Chapter 10
This lab integrates concepts from **Sections 10.2-10.4**:
- Environmental pressures on reproductive strategy (Table 10.1)
- External vs internal fertilization trade-offs
- Parental investment patterns
- r-selection (many offspring, low care) vs K-selection (few offspring, high care)

### 🥚 The Big Question
**Why do cod produce 9 million eggs while elephants have one baby?**  
Both strategies work - but why? Let's calculate the trade-offs!

In [ ]:
# === GOOGLE COLAB SETUP - RUN THIS FIRST ===
try:
    from google.colab import output
    output.enable_custom_widget_manager()
    print("✓ Widgets enabled")
except:
    print("✓ Running outside Colab")

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import *
from IPython.display import display, clear_output
from datetime import datetime
import os

print("✓ Ready to explore reproductive strategies!")

## Part 1: The r vs K Spectrum

### Life History Theory
Every organism faces fundamental trade-offs in how to allocate limited resources:

### r-Selection (Quantity Strategy)
- **Many offspring**
- **Small investment per offspring**
- **Little/no parental care**
- **Early maturation**
- **Best for:** Unpredictable environments, high mortality

**Examples:** Cod (9 million eggs), Frogs (thousands of eggs), Oysters

### K-Selection (Quality Strategy)
- **Few offspring**
- **Large investment per offspring**
- **Extensive parental care**
- **Delayed maturation**
- **Best for:** Stable environments, competitive conditions

**Examples:** Elephants (1 calf), Albatross (1 egg), Humans

### The Fitness Equation
**Reproductive fitness** = (# offspring) × (survival probability)

$$F = n \times s$$

Where:
- F = fitness (offspring surviving to reproduce)
- n = number of offspring
- s = survival probability per offspring

### The Trade-off
**More offspring → Less investment each → Lower survival**

$$s = s_{max} \times e^{-kn}$$

Where:
- s_max = maximum survival (with infinite investment)
- k = trade-off constant (how fast survival drops)
- n = number of offspring

### From Chapter 10 (Table 10.1)
| Environment | Strategy | Examples |
|-------------|----------|----------|
| Unpredictable | High quantity, low quality | Marine fish, amphibians |
| Stable | Low quantity, high quality | Large mammals, raptors |
| Seasonal | Synchronized breeding | Migratory birds |
| Predation pressure | Rapid reproduction | Small mammals, prey fish |

In [ ]:
# Fitness calculation functions
def calculate_survival(n_offspring, s_max=0.95, k=0.15):
    """
    Calculate per-offspring survival probability
    
    Trade-off: more offspring = less investment each = lower survival
    
    Parameters:
    - n_offspring: Number of offspring
    - s_max: Maximum survival (with infinite investment)
    - k: Trade-off steepness (higher = steeper trade-off)
    
    Returns:
    - Survival probability (0-1)
    """
    survival = s_max * np.exp(-k * n_offspring)
    return max(0.001, min(0.99, survival))  # Keep realistic bounds

def calculate_fitness(n_offspring, s_max=0.95, k=0.15):
    """
    Calculate reproductive fitness
    
    Fitness = offspring number × survival probability
    """
    survival = calculate_survival(n_offspring, s_max, k)
    fitness = n_offspring * survival
    return fitness, survival

def optimal_clutch_size(s_max=0.95, k=0.15):
    """
    Find optimal offspring number (maximizes fitness)
    """
    n_range = np.linspace(1, 20, 1000)
    fitness_vals = [calculate_fitness(n, s_max, k)[0] for n in n_range]
    optimal_n = n_range[np.argmax(fitness_vals)]
    max_fitness = max(fitness_vals)
    return optimal_n, max_fitness

# Test examples
print("EXAMPLES:")
print("="*70)

# r-selected (many offspring)
cod_n = 1000  # Simplified from millions
cod_fit, cod_surv = calculate_fitness(cod_n, s_max=0.05, k=0.0001)
print(f"\nCOD (r-selected):")
print(f"  Offspring: {cod_n:,}")
print(f"  Survival: {cod_surv:.4f} ({cod_surv*100:.2f}%)")
print(f"  Fitness: {cod_fit:.1f} offspring survive")

# K-selected (few offspring)
elephant_n = 1
elephant_fit, elephant_surv = calculate_fitness(elephant_n, s_max=0.85, k=0.05)
print(f"\nELEPHANT (K-selected):")
print(f"  Offspring: {elephant_n}")
print(f"  Survival: {elephant_surv:.4f} ({elephant_surv*100:.2f}%)")
print(f"  Fitness: {elephant_fit:.2f} offspring survive")

# Intermediate
opt_n, opt_fit = optimal_clutch_size(s_max=0.7, k=0.2)
print(f"\nOPTIMAL STRATEGY (moderate environment):")
print(f"  Optimal offspring: {opt_n:.1f}")
print(f"  Maximum fitness: {opt_fit:.2f}")

print("\n✓ Calculation functions ready!")

## Part 2: Species Database

Real data from Chapter 10 and scientific literature:

In [ ]:
# Reproductive strategy database
species_reproductive = {
    'Cod': {
        'offspring': 9000000,  # Millions!
        'offspring_size_g': 0.001,  # 1mg egg
        'parental_care_days': 0,
        'survival_rate': 0.00001,  # 0.001%
        'maturity_years': 2,
        'strategy': 'Extreme r',
        'environment': 'Unpredictable ocean',
        'desc': 'Ultimate r-strategist. Releases millions of tiny eggs into ocean. '
                'No parental care. Massive mortality but some survive.'
    },
    'Salmon': {
        'offspring': 4000,
        'offspring_size_g': 0.2,  # 200mg
        'parental_care_days': 0,
        'survival_rate': 0.001,  # 0.1%
        'maturity_years': 3,
        'strategy': 'High r',
        'environment': 'Seasonal rivers',
        'desc': 'Semelparous (breed once, die). Buries eggs in gravel. '
                'No care after spawning. Homing behavior for optimal sites.'
    },
    'Frog': {
        'offspring': 2000,
        'offspring_size_g': 0.005,
        'parental_care_days': 0,
        'survival_rate': 0.01,  # 1%
        'maturity_years': 1,
        'strategy': 'High r',
        'environment': 'Unpredictable wetlands',
        'desc': 'External fertilization. Thousands of eggs in water. '
                'Tadpoles fend for themselves. High predation.'
    },
    'Poison Dart Frog': {
        'offspring': 6,
        'offspring_size_g': 0.01,
        'parental_care_days': 60,
        'survival_rate': 0.70,  # 70%!
        'maturity_years': 1,
        'strategy': 'Moderate K',
        'environment': 'Stable rainforest',
        'desc': 'Intensive biparental care! Both parents transport tadpoles individually. '
                'Mother feeds unfertilized eggs. High offspring survival.'
    },
    'Sea Turtle': {
        'offspring': 110,
        'offspring_size_g': 40,
        'parental_care_days': 0,
        'survival_rate': 0.001,  # 0.1%
        'maturity_years': 25,
        'strategy': 'Moderate r',
        'environment': 'Marine with terrestrial nesting',
        'desc': 'Returns to natal beach. Digs nest, lays 50-200 eggs. '
                'Temperature determines sex. No parental care. Long maturation.'
    },
    'Stickleback': {
        'offspring': 200,
        'offspring_size_g': 0.002,
        'parental_care_days': 10,
        'survival_rate': 0.15,  # 15%
        'maturity_years': 1,
        'strategy': 'Moderate',
        'environment': 'Seasonal waters',
        'desc': 'Male builds nest and guards eggs. Fans water for oxygen. '
                'Guards fry. Moderate care increases survival.'
    },
    'Duck': {
        'offspring': 12,
        'offspring_size_g': 50,
        'parental_care_days': 60,
        'survival_rate': 0.50,
        'maturity_years': 1,
        'strategy': 'Moderate K',
        'environment': 'Seasonal wetlands',
        'desc': 'Precocial young (mobile at hatching). Mother leads to food. '
                'Moderate care for 2 months. Good survival.'
    },
    'Emperor Penguin': {
        'offspring': 1,
        'offspring_size_g': 450,
        'parental_care_days': 150,
        'survival_rate': 0.60,
        'maturity_years': 5,
        'strategy': 'High K',
        'environment': 'Extreme Antarctic',
        'desc': 'Single egg. Male incubates 64 days without eating! '
                'Both parents feed chick for months. Extreme parental investment.'
    },
    'Albatross': {
        'offspring': 1,
        'offspring_size_g': 280,
        'parental_care_days': 280,
        'survival_rate': 0.75,
        'maturity_years': 10,
        'strategy': 'Extreme K',
        'environment': 'Oceanic islands',
        'desc': 'Single egg every 2 years. 9 months of care! Both parents feed. '
                'Chick depends on parents completely. Very long life.'
    },
    'Mouse': {
        'offspring': 7,
        'offspring_size_g': 1,
        'parental_care_days': 21,
        'survival_rate': 0.40,
        'maturity_years': 0.15,  # 2 months
        'strategy': 'Moderate r',
        'environment': 'Unstable/predation',
        'desc': 'Multiple litters per year. Rapid reproduction. '
                'Short care period. Early maturity. High predation.'
    },
    'Elephant': {
        'offspring': 1,
        'offspring_size_g': 120000,  # 120kg!
        'parental_care_days': 3650,  # 10 years!
        'survival_rate': 0.85,
        'maturity_years': 14,
        'strategy': 'Extreme K',
        'environment': 'Stable savanna',
        'desc': 'Single calf. 22-month gestation! Years of maternal care. '
                'Social learning critical. One offspring every 4-5 years.'
    },
    'Human': {
        'offspring': 1,
        'offspring_size_g': 3500,
        'parental_care_days': 6570,  # 18 years!
        'survival_rate': 0.98,  # Modern
        'maturity_years': 18,
        'strategy': 'Extreme K',
        'environment': 'Complex social',
        'desc': 'Single births typical. Extremely long childhood. '
                'Extended learning period. Cooperative care. Cultural transmission.'
    }
}

# Calculate fitness for all
for name in species_reproductive:
    sp = species_reproductive[name]
    sp['fitness'] = sp['offspring'] * sp['survival_rate']
    sp['total_investment_days'] = sp['parental_care_days'] * sp['offspring']

print("REPRODUCTIVE STRATEGY DATABASE")
print("="*90)
print(f"{'Species':<20}{'Offspring':<12}{'Care (days)':<15}{'Survival':<12}{'Fitness':<10}")
print("="*90)

# Sort by strategy
for strat in ['Extreme r', 'High r', 'Moderate r', 'Moderate', 'Moderate K', 'High K', 'Extreme K']:
    for name in sorted(species_reproductive.keys()):
        sp = species_reproductive[name]
        if sp['strategy'] == strat:
            print(f"{name:<20}{sp['offspring']:<12,}{sp['parental_care_days']:<15}"
                  f"{sp['survival_rate']:<12.2%}{sp['fitness']:<10.1f}")

print("="*90)
print("\n✓ Database ready!")

## Part 3: Interactive Strategy Calculator

In [ ]:
def analyze_strategy(species_name):
    """Analyze reproductive strategy"""
    
    sp = species_reproductive[species_name]
    
    # Create 4-panel viz
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Reproductive Fitness', 'r-K Spectrum Position',
                       'Trade-off: Offspring vs Survival', 'Investment Pattern'),
        specs=[[{'type': 'indicator'}, {'type': 'indicator'}],
               [{'type': 'scatter'}, {'type': 'bar'}]]
    )
    
    # 1. Fitness gauge
    fig.add_trace(go.Indicator(
        mode="number+gauge",
        value=sp['fitness'],
        title={'text': "Expected Survivors"},
        gauge={
            'axis': {'range': [0, 100]},
            'bar': {'color': '#E74C3C'},
            'steps': [
                {'range': [0, 1], 'color': '#ECF0F1'},
                {'range': [1, 10], 'color': '#D5DBDB'},
                {'range': [10, 50], 'color': '#AED6F1'},
                {'range': [50, 100], 'color': '#85C1E9'}
            ]
        }
    ), row=1, col=1)
    
    # 2. r-K position
    # Calculate position (0=extreme r, 100=extreme K)
    log_offspring = np.log10(max(sp['offspring'], 1))
    rk_score = 100 * (1 - (log_offspring / 7))  # 0-100 scale
    
    fig.add_trace(go.Indicator(
        mode="number+gauge+delta",
        value=rk_score,
        title={'text': sp['strategy']},
        gauge={
            'axis': {'range': [0, 100]},
            'bar': {'color': '#2ECC71'},
            'steps': [
                {'range': [0, 30], 'color': '#FFE5E5', 'name': 'r'},
                {'range': [30, 70], 'color': '#FFF9E5'},
                {'range': [70, 100], 'color': '#E5F5FF', 'name': 'K'}
            ],
            'threshold': {'line': {'color': 'black', 'width': 2}, 'value': 50}
        },
        delta={'reference': 50}
    ), row=1, col=2)
    
    # 3. Trade-off scatter - compare all species
    for sp_name, sp_data in species_reproductive.items():
        is_current = (sp_name == species_name)
        fig.add_trace(go.Scatter(
            x=[sp_data['offspring']],
            y=[sp_data['survival_rate']],
            mode='markers+text' if is_current else 'markers',
            name=sp_name,
            marker=dict(
                size=15 if is_current else 8,
                color='#E74C3C' if is_current else '#95A5A6',
                symbol='star' if is_current else 'circle'
            ),
            text=[sp_name] if is_current else None,
            textposition='top center',
            showlegend=False,
            hovertemplate=f"{sp_name}<br>Offspring: {sp_data['offspring']:,}<br>Survival: {sp_data['survival_rate']:.2%}<extra></extra>"
        ), row=2, col=1)
    
    # 4. Investment breakdown
    categories = ['Per Offspring\nCare (days)', 'Total Investment\n(days)', 
                  'Maturity\n(years)']
    values = [sp['parental_care_days'], sp['total_investment_days'], 
              sp['maturity_years']]
    
    # Normalize for visualization
    max_care = max(s['parental_care_days'] for s in species_reproductive.values())
    max_total = max(s['total_investment_days'] for s in species_reproductive.values())
    max_mat = max(s['maturity_years'] for s in species_reproductive.values())
    
    normalized = [
        100 * sp['parental_care_days'] / max_care,
        100 * sp['total_investment_days'] / max_total,
        100 * sp['maturity_years'] / max_mat
    ]
    
    fig.add_trace(go.Bar(
        x=categories,
        y=normalized,
        marker_color=['#3498DB', '#9B59B6', '#F39C12'],
        text=[f"{v:.0f}" if v < 365 else f"{v/365:.1f}y" for v in values],
        textposition='outside',
        showlegend=False
    ), row=2, col=2)
    
    # Update axes
    fig.update_xaxes(title_text="Offspring Number", type="log", row=2, col=1)
    fig.update_yaxes(title_text="Survival Rate", row=2, col=1)
    fig.update_yaxes(title_text="Relative Scale", row=2, col=2)
    
    fig.update_layout(
        height=700,
        title_text=f"<b>{species_name} Reproductive Strategy</b>"
    )
    
    # Print summary
    print("\n" + "="*70)
    print(f"STRATEGY ANALYSIS: {species_name}")
    print("="*70)
    print(f"Classification: {sp['strategy']}")
    print(f"Environment: {sp['environment']}")
    print(f"\nREPRODUCTIVE PARAMETERS:")
    print(f"  Offspring per reproductive event: {sp['offspring']:,}")
    print(f"  Offspring size: {sp['offspring_size_g']:.3f}g")
    print(f"  Parental care: {sp['parental_care_days']} days")
    print(f"  Survival rate: {sp['survival_rate']:.2%}")
    print(f"  Time to maturity: {sp['maturity_years']} years")
    print(f"\nFITNESS CALCULATION:")
    print(f"  Fitness = {sp['offspring']:,} × {sp['survival_rate']:.2%} = {sp['fitness']:.1f}")
    print(f"  Expected survivors: ~{sp['fitness']:.0f} offspring reach reproduction")
    print(f"\n{sp['desc']}")
    print("="*70)
    
    fig.show()

species_dropdown = Dropdown(
    options=sorted(species_reproductive.keys(), 
                  key=lambda x: species_reproductive[x]['offspring']),
    value='Salmon',
    description='Species:',
    style={'description_width': '80px'}
)

display(HTML("<h3>🥚 Reproductive Strategy Analyzer</h3>"))
interact(analyze_strategy, species_name=species_dropdown);

## Part 4: Trade-off Visualizer

In [ ]:
def visualize_tradeoffs():
    """Visualize all species on r-K spectrum"""
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Offspring Number vs Survival', 'Investment vs Fitness',
                       'r-K Spectrum', 'Strategy by Environment'),
        specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
               [{'type': 'bar'}, {'type': 'bar'}]]
    )
    
    # Prepare data
    species_names = list(species_reproductive.keys())
    
    # 1. Main trade-off
    for name in species_names:
        sp = species_reproductive[name]
        fig.add_trace(go.Scatter(
            x=[sp['offspring']],
            y=[sp['survival_rate']],
            mode='markers+text',
            text=[name],
            textposition='top center',
            marker=dict(size=12),
            name=name,
            showlegend=False
        ), row=1, col=1)
    
    # 2. Investment vs fitness
    investments = [species_reproductive[n]['parental_care_days'] for n in species_names]
    fitnesses = [species_reproductive[n]['fitness'] for n in species_names]
    
    fig.add_trace(go.Scatter(
        x=investments,
        y=fitnesses,
        mode='markers+text',
        text=species_names,
        textposition='top center',
        marker=dict(size=10, color='#E74C3C'),
        showlegend=False
    ), row=1, col=2)
    
    # 3. r-K spectrum bar
    offspring_counts = [species_reproductive[n]['offspring'] for n in species_names]
    sorted_indices = np.argsort(offspring_counts)[::-1]  # High to low
    sorted_names = [species_names[i] for i in sorted_indices]
    sorted_counts = [offspring_counts[i] for i in sorted_indices]
    
    colors_rk = ['#FF6B6B' if c > 1000 else '#FFA07A' if c > 100 else 
                 '#FFD93D' if c > 10 else '#6BCB77' for c in sorted_counts]
    
    fig.add_trace(go.Bar(
        x=sorted_names,
        y=sorted_counts,
        marker_color=colors_rk,
        showlegend=False
    ), row=2, col=1)
    
    # 4. Environment categories
    envs = {}
    for name, sp in species_reproductive.items():
        env_type = sp['environment'].split()[0]  # First word
        if env_type not in envs:
            envs[env_type] = []
        envs[env_type].append(sp['fitness'])
    
    env_names = list(envs.keys())
    avg_fitness = [np.mean(envs[e]) for e in env_names]
    
    fig.add_trace(go.Bar(
        x=env_names,
        y=avg_fitness,
        marker_color='#3498DB',
        text=[f"{f:.1f}" for f in avg_fitness],
        textposition='outside',
        showlegend=False
    ), row=2, col=2)
    
    # Update layout
    fig.update_xaxes(title_text="Offspring #", type="log", row=1, col=1)
    fig.update_yaxes(title_text="Survival Rate", row=1, col=1)
    fig.update_xaxes(title_text="Care (days)", row=1, col=2)
    fig.update_yaxes(title_text="Fitness", row=1, col=2)
    fig.update_yaxes(title_text="Offspring #", type="log", row=2, col=1)
    fig.update_xaxes(tickangle=45, row=2, col=1)
    fig.update_yaxes(title_text="Avg Fitness", row=2, col=2)
    
    fig.update_layout(
        height=800,
        title_text='<b>Complete r-K Strategy Comparison</b>'
    )
    
    print("\nSTRATEGY COMPARISON")
    print("="*70)
    print(f"{'Species':<20}{'Offspring':<12}{'Survival':<12}{'Fitness':<10}")
    print("="*70)
    for name in sorted_names:
        sp = species_reproductive[name]
        print(f"{name:<20}{sp['offspring']:<12,}{sp['survival_rate']:<12.2%}{sp['fitness']:<10.1f}")
    print("="*70)
    
    fig.show()

compare_btn = Button(
    description='📊 Compare All',
    button_style='info',
    layout={'width': '180px'}
)

output_compare = Output()

def on_compare(b):
    with output_compare:
        clear_output(wait=True)
        visualize_tradeoffs()

compare_btn.on_click(on_compare)

display(HTML("<h3>📊 Complete Trade-off Analysis</h3>"))
display(compare_btn)
display(output_compare)

## Part 5: Challenge Problems

### Challenge 1: Optimal Clutch Size 🥚

**Scenario**: You're a stickleback fish. Environmental conditions affect survival.

**Given**:
- Good year: s_max = 0.50, k = 0.10
- Average year: s_max = 0.30, k = 0.15
- Bad year: s_max = 0.15, k = 0.20

**Questions**:
1. What's the optimal clutch size for each condition?
2. How does fitness change?
3. Should you adjust clutch size or keep it constant?

<details>
<summary>Solution</summary>

**Calculations** (using optimal_clutch_size function):
- Good year: ~10 eggs optimal, fitness ≈ 1.8
- Average year: ~6 eggs optimal, fitness ≈ 1.2
- Bad year: ~5 eggs optimal, fitness ≈ 0.75

**Insight**: Optimal clutch decreases in bad years!
- Spreading resources thin hurts survival even more
- Better to invest more per offspring when conditions tough

**Real sticklebacks**: Adjust clutch size based on conditions! Evidence of adaptive plasticity.
</details>

In [ ]:
# Challenge 1 workspace
print("CHALLENGE 1: OPTIMAL CLUTCH SIZE")
print("="*70)

conditions = {
    'Good year': {'s_max': 0.50, 'k': 0.10},
    'Average year': {'s_max': 0.30, 'k': 0.15},
    'Bad year': {'s_max': 0.15, 'k': 0.20}
}

print(f"\n{'Condition':<15}{'Optimal Clutch':<18}{'Max Fitness':<15}{'Strategy'}")
print("-"*70)

for cond_name, params in conditions.items():
    opt_n, max_fit = optimal_clutch_size(params['s_max'], params['k'])
    strategy = "More eggs" if opt_n > 6 else "Fewer eggs, more care"
    print(f"{cond_name:<15}{opt_n:<18.1f}{max_fit:<15.2f}{strategy}")

print("\n" + "="*70)
print("KEY INSIGHT:")
print("In bad years, produce FEWER offspring but invest more in each!")
print("Quality over quantity when survival is already low.")
print("="*70)

### Challenge 2: The Cod Paradox 🐟

**Question**: Cod produce 9 million eggs. Even with 0.001% survival, that's 90 survivors. Why not fewer eggs with better care?

**Consider**:
- Ocean is unpredictable
- Can't predict where food/currents will be
- Parental care impossible in open ocean
- Broadcasting strategy: some eggs will hit jackpot

**Calculate**: If cod tried K-strategy (10 eggs, 50% survival):
- Fitness = 10 × 0.50 = 5 survivors
- BUT: Would require internal fertilization, protection, feeding
- Not feasible in open ocean!

**Model it**: What if cod survival increased 10× with parental care?

<details>
<summary>Solution</summary>

**Current r-strategy**:
- 9 million eggs × 0.00001 = 90 survivors

**Hypothetical K-strategy**:
- Could do 1000 eggs with 0.01% survival (10×) = 10 survivors
- Or 100 eggs with 0.1% survival (100×) = 10 survivors
- Or 10 eggs with 1% survival (1000×) = 10 survivors

**Problem**: Getting 10-1000× survival boost requires:
- Protection from predators
- Reliable food source
- Stable environment

**None feasible in open ocean!**

**Lesson**: Environment constrains strategy. r-selection isn't "worse" - it's optimal for unpredictable habitats!
</details>

### Challenge 3: Design a Species 🧬

**Scenario**: You're evolution. Design optimal reproductive strategy for:

**Environment A**: Tropical coral reef
- Stable temperature
- Predictable food
- High predation
- Limited safe sites

**Environment B**: Arctic tundra
- Extreme seasonal variation
- Summer food abundance, winter scarcity
- Low predation
- Short breeding season

**Design parameters**:
1. Offspring number
2. Parental care duration
3. Offspring size
4. Reproductive timing

<details>
<summary>Solution</summary>

**Environment A (Coral Reef)**:
- Moderate offspring (10-50)
- HIGH parental care (guard eggs/young)
- Small-medium size
- Year-round breeding possible
- **Why**: Stable environment allows care. High predation demands protection. Limited safe sites favor defense of territory.
- **Real examples**: Clownfish (guard eggs), Damselfish

**Environment B (Arctic)**:
- Moderate offspring (3-8)
- Moderate care but TIMING critical
- Large offspring (energy stores)
- Synchronized spring breeding
- **Why**: Must time birth for summer food peak. Large offspring survive better. Can't care year-round. Seasonal synchronization essential.
- **Real examples**: Arctic ground squirrel, Caribou

**Key principle**: Strategy must match environment constraints!
</details>

## Part 6: Export Your Work

In [ ]:
def export_results():
    """Export reproductive strategy analysis"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = "/content"
    
    # Create summary
    data = []
    for name, sp in species_reproductive.items():
        data.append({
            'Species': name,
            'Strategy': sp['strategy'],
            'Offspring': sp['offspring'],
            'Offspring_Size_g': sp['offspring_size_g'],
            'Care_Days': sp['parental_care_days'],
            'Survival_Rate': sp['survival_rate'],
            'Fitness': sp['fitness'],
            'Maturity_Years': sp['maturity_years'],
            'Environment': sp['environment']
        })
    
    df = pd.DataFrame(data).sort_values('Offspring', ascending=False)
    
    # Save CSV
    csv_file = f"{output_dir}/lab_10_1_reproductive_strategies_{timestamp}.csv"
    df.to_csv(csv_file, index=False)
    print(f"✓ Data saved: {csv_file}")
    
    # Create viz
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('r-K Spectrum', 'Fitness Comparison')
    )
    
    fig.add_trace(go.Bar(
        x=df['Species'],
        y=df['Offspring'],
        marker_color='#E74C3C',
        showlegend=False
    ), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=df['Species'],
        y=df['Fitness'],
        marker_color='#2ECC71',
        text=[f"{f:.1f}" for f in df['Fitness']],
        textposition='outside',
        showlegend=False
    ), row=1, col=2)
    
    fig.update_xaxes(tickangle=45, row=1, col=1)
    fig.update_xaxes(tickangle=45, row=1, col=2)
    fig.update_yaxes(title_text="Offspring #", type="log", row=1, col=1)
    fig.update_yaxes(title_text="Fitness", row=1, col=2)
    
    fig.update_layout(
        height=500,
        title_text='<b>Lab 10.1 Summary: Reproductive Strategies</b>'
    )
    
    html_file = f"{output_dir}/lab_10_1_summary_{timestamp}.html"
    fig.write_html(html_file)
    print(f"✓ Visualization saved: {html_file}")
    
    try:
        png_file = f"{output_dir}/lab_10_1_summary_{timestamp}.png"
        fig.write_image(png_file, width=1200, height=600)
        print(f"✓ PNG saved: {png_file}")
    except:
        print("ℹ PNG requires kaleido")
    
    print("\n" + "="*60)
    print("EXPORT COMPLETE!")
    print("="*60)
    print(f"Files in: {output_dir}")
    print("To download: Click 📁 → /content → right-click → Download")
    print("✓ Ready!")

export_btn = Button(
    description='📥 Export Results',
    button_style='success',
    icon='download',
    layout={'width': '200px'}
)

export_btn.on_click(lambda b: export_results())

display(HTML("<h3>📤 Export Your Work</h3>"))
display(export_btn)

## Summary

### What You've Learned

✅ **r vs K strategies** - Quantity vs quality trade-offs  
✅ **Fitness calculations** - Offspring × survival  
✅ **Environmental matching** - Strategy depends on habitat  
✅ **Optimal clutch size** - Adjusts with conditions  
✅ **Life history theory** - Allocation of limited resources  

### Key Insights

**From the data**:
- Cod: 9 million eggs, 0.001% survival = 90 survivors
- Elephant: 1 calf, 85% survival = 0.85 survivors
- Both strategies work! Different environments, different solutions

**Trade-offs discovered**:
1. More offspring → Less investment each → Lower survival
2. Stable environment → K-selection pays off
3. Unpredictable environment → r-selection optimal
4. Parental care increases survival but limits offspring number

### Connection to Chapter 10

This lab illustrated **Sections 10.2-10.4**:
- Table 10.1: Environmental strategies
- External vs internal fertilization trade-offs
- Parental investment patterns
- Life history diversity across vertebrates

### Real-World Applications

- **Conservation**: Predict species vulnerability
- **Fisheries**: Understand population dynamics
- **Agriculture**: Breeding program design
- **Medicine**: Life history effects on health
- **Evolution**: Predict adaptive responses

### The Big Picture

There's no single "best" strategy:
- **r-selection** works for unstable environments
- **K-selection** works for stable, competitive environments
- **Most species** are intermediate
- **Flexibility** allows adjustment to conditions

Evolution optimizes for environment, not perfection!

**Congratulations on completing Lab 10.1!** 🎉

You now understand why a cod and an elephant both succeed with opposite strategies!